In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math
import torch.nn
import torch.optim
import torch.utils.data

### Display DataFrame

In [ ]:
# Your code here
import pandas as pd
# Google Drive link incase Github link doesn't work
GDrive_Link = "https://drive.google.com/file/d/1ZJP3osriK0isJDyBlyqIhbZ1gvT2N4aH/view?usp=sharing"
print("Google Drive link incase GitHub link doesn't work:")
print(GDrive_Link)

# Import CSV using raw Github link
Github_Link = "https://raw.githubusercontent.com/khangnngo12/Kinematics-of-M33/main/Redshift.csv"
df = pd.read_csv(Github_Link)
print("CSV file containing velocities (km/s), RA, Dec, and other informations:")
df
#raise NotImplementedError()

### Display Velocity Map

This section will display the velocity map with a fitted ellipse and two parabolas. The data points are created using RA and Dec, color-coded using its measured H-alpha velocities. The parabolas will divide the map into three areas: North, South, and Center. 

In [ ]:
center_RA = (1 + (33/60) + (51.75/3600))*15 #in deg
center_Dec = (30 + (39/60) + (36.630/3600)) # in deg

In [ ]:
def linear_func(x,m,b): #y=mx+b function
    return (m*x)+b

In [ ]:
def classification_func(RA,Dec):
    
    """
    Function to classified each point into North, South, and Center.
    
    Parameters
    ----------
    RA: ndarray,req
        Containing RA values used in map
    Dec: ndarray, req
        Containing Dec values used in map
        
    Returns
    ----------
    NorthParabola: ndarray containing bools. True == Inside North parabola
    SouthParabola: ndarray containing bools. True == Inside South parabola
    Center: ndarray containing bools. True == Inside Center region
    """
    
    import cmath
    theta = np.radians(-32)
    
    Dec_Val_NP_List = [] #list containing Dec values along North parabola
    Dec_Val_SP_List = [] #list containing Dec values along South parabola
    
    for RA_Val in RA:
    
        # find a,b,c for quadratic formula
        a_NorthPara = -0.5*np.sin(theta)
        b_NorthPara = 0.5*np.cos(theta)
        c_NorthPara = NorthPara_XCoor - RA_Val
        
        a_SouthPara = 0.5*np.sin(theta)
        b_SouthPara = -0.5*np.cos(theta)
        c_SouthPara = SouthPara_XCoor - RA_Val
        
        # calculate the discriminan
        d_NorthPara = (b_NorthPara**2) - (4*a_NorthPara*c_NorthPara) 
        d_SouthPara = (b_SouthPara**2) - (4*a_SouthPara*c_SouthPara)
        
        t_NorthPara = (-b_NorthPara + cmath.sqrt(d_NorthPara))/(2*a_NorthPara) # find t for North Parabola (Notice the plus)
        t_SouthPara = (-b_SouthPara - cmath.sqrt(d_SouthPara))/(2*a_SouthPara) # find t for South Parabola (Notice the subtract)
        
        Dec_Val_NorthPara = 0.5*(t_NorthPara*np.sin(theta) + (t_NorthPara**2)*np.cos(theta)) + NorthPara_YCoor #find corresponding Dec value using t
        Dec_Val_NP_List.append(Dec_Val_NorthPara)
        
        Dec_Val_SouthPara = -0.5*(t_SouthPara*np.sin(theta) + (t_SouthPara**2)*np.cos(theta)) + SouthPara_YCoor
        Dec_Val_SP_List.append(Dec_Val_SouthPara)
        
    NorthParabola = np.array(Dec-np.array(Dec_Val_NP_List) > 0)
    SouthParabola = np.array(Dec-np.array(Dec_Val_SP_List) < 0)
    Center = []
    for index in range(len(RA)):
        if NorthParabola[index] == False:
            if SouthParabola[index] == False:
                Center.append(True)
            else:
                Center.append(False)
        else:
            Center.append(False)
    
    return NorthParabola,SouthParabola,np.array(Center)

In [ ]:
import math
from math import pi, cos, sin

# Get datapoints from CSV files. 
df_qop2_sorted = df["QOP"] >= 2

fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)")
ax.set_ylabel("Declination (deg)")
ax.set_title("M33 Velocity Color Map (QOP >= 2)")
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,30.9)
ax.invert_xaxis()
ax.set_aspect('equal')
cc_map = ax.scatter(df["RA"][df_qop2_sorted]
                    ,df["DEC"][df_qop2_sorted]
                    ,c=df["Velocity (km/s)"][df_qop2_sorted],s=7,cmap='magma')
plt.colorbar(cc_map,label="Velocity (km/s)")
fig.set_figwidth(5)
fig.set_figheight(5)
plt.grid(True,linestyle="--")

#------------------------------

#Fit Ellipse
u= center_RA       #x-position of the center
v= center_Dec    #y-position of the center
a= 0.52     #radius on the x-axis
b= 0.32      #radius on the y-axis
t_rot=math.radians(58) #rotation angle

t = np.linspace(0, 2*pi, 100)
Ell = np.array([a*np.cos(t) , b*np.sin(t)])  
     #u,v removed to keep the same center location
R_rot = np.array([[cos(t_rot) , -sin(t_rot)],[sin(t_rot) , cos(t_rot)]])  
     #2-D rotation matrix

Ell_rot = np.zeros((2,Ell.shape[1]))
for i in range(Ell.shape[1]):
    Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

plt.plot(u+Ell_rot[0,:] , v+Ell_rot[1,:],c='black',linewidth=0.8)    #rotated ellipse

#Semi-minor Axis
x_SMi_0 = center_RA + (b*cos(np.radians(32))) #x-coor of left Semi-minor axis point 
y_SMi_0 = center_Dec - (b*sin(np.radians(32))) #y-coor of left Semi-minor axis point
x_SMi_1 = center_RA - (b*cos(np.radians(32))) #x-coor of right Semi-minor axis point
y_SMi_1 = center_Dec + (b*sin(np.radians(32))) #y-coor of right Semi-minor axis point
plt.scatter(x_SMi_0,y_SMi_0,s=15,c="black")
plt.scatter(x_SMi_1,y_SMi_1,s=15,c="black")

m_SMi = (y_SMi_1-y_SMi_0)/(x_SMi_1-x_SMi_0) #slope of line through Semi-minor axis 
b_SMi = y_SMi_0 - (m_SMi*x_SMi_0) #y-inter of line through Semi-minor axis
x_SMi = np.linspace(x_SMi_1,x_SMi_0,1000) #x values of line through Semi-minor axis
y_SMi = linear_func(x_SMi,m_SMi,b_SMi) #y values of line through Semi-minor axis
plt.plot(x_SMi,y_SMi,c='black',linewidth=0.8,linestyle="--")

#Semi-major Axis
x_SouthSMa = center_RA - (a*np.sin(np.radians(32))) #x-coor of North Semi-major axis point
y_SouthSMa = center_Dec - (a*np.cos(np.radians(32))) #y-coor of Nouth Semi-major axis point
x_NorthSMa = center_RA + (a*np.sin(np.radians(32))) #x-coor of South Semi-major axis point
y_NorthSMa = center_Dec + (a*np.cos(np.radians(32))) #y-coor of South Semi-major axis point
plt.scatter(x_NorthSMa,y_NorthSMa,c="black",s=15)
plt.scatter(x_SouthSMa,y_SouthSMa,c="black",s=15)

m_SMa = (y_NorthSMa-y_SouthSMa)/(x_NorthSMa-x_SouthSMa) #slope of line through Semi-major axis
b_SMa = y_SouthSMa - (m_SMa*x_SouthSMa) #y-inter of line through Semi-major axis 
x_SMa = np.linspace(x_NorthSMa,x_SouthSMa,1000) #x values of line through Semi-major axis
y_SMa = linear_func(x_SMa,m_SMa,b_SMa) #y values of line through Semi-major axis
plt.plot(x_SMa,y_SMa,c='black',linewidth=0.8,linestyle="--")

NorthPara_XCoor = x_NorthSMa - ((19*a/20)*np.sin(np.radians(32))) #coordinates to move North parabola to
NorthPara_YCoor = linear_func(NorthPara_XCoor,m_SMa,b_SMa)

SouthPara_XCoor = x_SouthSMa + ((19*a/20)*np.sin(np.radians(32))) #coordinate to move South parabola to
SouthPara_YCoor = linear_func(SouthPara_XCoor,m_SMa,b_SMa)

#Fit Two Parabolas
theta = np.radians(-32) #degrees to tilt parabolas
t = np.linspace(-10,10,1000)
#x = (0.5*t)*np.cos(theta) - (t**2)*np.sin(theta) + x_2
#y = (0.5*t)*np.sin(theta) + (t**2)*np.cos(theta) + y_2
x_NorthPara = 0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + NorthPara_XCoor #x values of North parabola
y_NorthPara = 0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + NorthPara_YCoor #y values of North parabola
plt.plot(x_NorthPara,y_NorthPara,c="blue",linewidth=1,linestyle="--")
x_SouthPara = -0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + SouthPara_XCoor #x values of South parabola
y_SouthPara = -0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + SouthPara_YCoor #y values of South parabola
plt.plot(x_SouthPara,y_SouthPara,c="blue",linewidth=1,linestyle="--")

fig.set_figwidth(5)
fig.set_figheight(5)

The codes below will classified each datapoint into one of the three regions. After it is all sorted, is plotted again but this time color-coded based on which region it belong to. 

In [ ]:
# Call classification function to sort each point.
NorthParabola,SouthParabola,Center = classification_func(df["RA"],df["DEC"])

In [ ]:
import math
from math import pi, cos, sin

#Display Data
fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)")
ax.set_ylabel("Declination (deg)")
ax.set_title("Color Coded Map")
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,31)
#ax.set_xlim(23,24)
#ax.set_ylim(30.2,31.2)
ax.invert_xaxis()
#ax.set_aspect(1/0.861)
ax.set_aspect("equal")
plt.grid(True,linestyle="--")

# Color code each regions
ax.scatter(df["RA"][NorthParabola],df["DEC"][NorthParabola],s=7,c="red",label="North")
ax.scatter(df["RA"][SouthParabola],df["DEC"][SouthParabola],s=7,c="blue",label="South")
ax.scatter(df["RA"][Center],df["DEC"][Center],s=7,c="green",label="Center")
plt.legend()

#-------------------------------------

#Fit Ellipse
u= center_RA       #x-position of the center
v= center_Dec    #y-position of the center
a= 0.52     #radius on the x-axis
b= 0.32      #radius on the y-axis
t_rot=math.radians(58) #rotation angle

t = np.linspace(0, 2*pi, 100)
Ell = np.array([a*np.cos(t) , b*np.sin(t)])  
     #u,v removed to keep the same center location
R_rot = np.array([[cos(t_rot) , -sin(t_rot)],[sin(t_rot) , cos(t_rot)]])  
     #2-D rotation matrix

Ell_rot = np.zeros((2,Ell.shape[1]))
for i in range(Ell.shape[1]):
    Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

plt.plot(u+Ell_rot[0,:] , v+Ell_rot[1,:],c='black',linewidth=0.8)    #rotated ellipse

#Semi-minor Axis
x_SMi_0 = center_RA + (b*cos(np.radians(32))) #x-coor of left Semi-minor axis point 
y_SMi_0 = center_Dec - (b*sin(np.radians(32))) #y-coor of left Semi-minor axis point
x_SMi_1 = center_RA - (b*cos(np.radians(32))) #x-coor of right Semi-minor axis point
y_SMi_1 = center_Dec + (b*sin(np.radians(32))) #y-coor of right Semi-minor axis point
plt.scatter(x_SMi_0,y_SMi_0,s=15,c="black")
plt.scatter(x_SMi_1,y_SMi_1,s=15,c="black")

m_SMi = (y_SMi_1-y_SMi_0)/(x_SMi_1-x_SMi_0) #slope of line through Semi-minor axis 
b_SMi = y_SMi_0 - (m_SMi*x_SMi_0) #y-inter of line through Semi-minor axis
x_SMi = np.linspace(x_SMi_1,x_SMi_0,1000) #x values of line through Semi-minor axis
y_SMi = linear_func(x_SMi,m_SMi,b_SMi) #y values of line through Semi-minor axis
plt.plot(x_SMi,y_SMi,c='black',linewidth=0.8,linestyle="--")

#Semi-major Axis
x_SouthSMa = center_RA - (a*np.sin(np.radians(32))) #x-coor of North Semi-major axis point
y_SouthSMa = center_Dec - (a*np.cos(np.radians(32))) #y-coor of Nouth Semi-major axis point
x_NorthSMa = center_RA + (a*np.sin(np.radians(32))) #x-coor of South Semi-major axis point
y_NorthSMa = center_Dec + (a*np.cos(np.radians(32))) #y-coor of South Semi-major axis point
plt.scatter(x_NorthSMa,y_NorthSMa,c="black",s=15)
plt.scatter(x_SouthSMa,y_SouthSMa,c="black",s=15)

m_SMa = (y_NorthSMa-y_SouthSMa)/(x_NorthSMa-x_SouthSMa) #slope of line through Semi-major axis
b_SMa = y_SouthSMa - (m_SMa*x_SouthSMa) #y-inter of line through Semi-major axis 
x_SMa = np.linspace(x_NorthSMa,x_SouthSMa,1000) #x values of line through Semi-major axis
y_SMa = linear_func(x_SMa,m_SMa,b_SMa) #y values of line through Semi-major axis
plt.plot(x_SMa,y_SMa,c='black',linewidth=0.8,linestyle="--")

NorthPara_XCoor = x_NorthSMa - ((19*a/20)*np.sin(np.radians(32))) #coordinates to move North parabola to
NorthPara_YCoor = linear_func(NorthPara_XCoor,m_SMa,b_SMa)

SouthPara_XCoor = x_SouthSMa + ((19*a/20)*np.sin(np.radians(32))) #coordinate to move South parabola to
SouthPara_YCoor = linear_func(SouthPara_XCoor,m_SMa,b_SMa)

#Fit Two Parabolas
theta = np.radians(-32) #degrees to tilt parabolas
t = np.linspace(-10,10,1000)
#x = (0.5*t)*np.cos(theta) - (t**2)*np.sin(theta) + x_2
#y = (0.5*t)*np.sin(theta) + (t**2)*np.cos(theta) + y_2
x_NorthPara = 0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + NorthPara_XCoor #x values of North parabola
y_NorthPara = 0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + NorthPara_YCoor #y values of North parabola
plt.plot(x_NorthPara,y_NorthPara,c="blue",linewidth=1,linestyle="--")
x_SouthPara = -0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + SouthPara_XCoor #x values of South parabola
y_SouthPara = -0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + SouthPara_YCoor #y values of South parabola
plt.plot(x_SouthPara,y_SouthPara,c="blue",linewidth=1,linestyle="--")

fig.set_figwidth(5)
fig.set_figheight(5)

### Histograms Using Velocity

In [ ]:
fig,ax = plt.subplots(3)
ax[0].hist(df["Velocity (km/s)"][NorthParabola])
ax[0].set_xlabel("Velocity (km/s)")

ax[1].hist(df["Velocity (km/s)"][SouthParabola])
ax[1].set_xlabel("Velocity (km/s)")

ax[2].hist(df["Velocity (km/s)"][Center])
ax[2].set_xlabel("Velocity (km/s)")

fig.tight_layout()

### Representing N,S,C as 0,1,2

In [ ]:
def y_train():
    
    """
    Assign each datapoint a value based on its location
    0 = North, 1 = Center, 2 = South
    
    Returns
    ----------
    y_train: ndarray containing 0,1,2 corresponing to zones.
    """
    
    y_train = []
    for index in np.arange(len(df["Velocity (km/s)"])):
        if index in np.where(NorthParabola==True)[0]:
            y_train.append(0)
        elif index in np.where(SouthParabola==True)[0]:
            y_train.append(2)
        else:
            y_train.append(1)
    
    return np.array(y_train)

In [ ]:
true_val = y_train() 

In [ ]:
print('Display TRUE results used to train the program:')
print(*true_val)

### NN

In [ ]:
#1688 datapoints
#1400 for training, 288 for testing
x_train = np.array([df["RA"][:1400],df["DEC"][:1400]]).T
y_train = np.array(true_val[:1400],dtype="float")
x_test = np.array([df["RA"][1400:1689],df["DEC"][1400:1869]]).T
y_test = np.array(true_val[1400:1689],dtype="float")

In [ ]:
x_train = torch.Tensor(x_train) #ndarray to tensor
y_train = torch.Tensor(y_train) #ndarray to tensor
x_test = torch.Tensor(x_test) #ndarray to tensor
y_test = torch.Tensor(y_test) #ndarray to tensor

In [ ]:
#Set up network, loss function, and optimizer
net = torch.nn.Sequential(torch.nn.Linear(2, 30),
    torch.nn.ReLU(),
    torch.nn.Linear(30, 40),
    torch.nn.ReLU(),
    torch.nn.Linear(40, 1),
) 

torch.save(net.state_dict(), 'net_FP.pth')

loss_func = torch.nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(),lr=0.05,momentum=0.9)
#loss_func = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(net.parameters(),lr=0.05)

In [ ]:
#Pack data
xy_train = torch.utils.data.TensorDataset(x_train, y_train)
xy_test = torch.utils.data.TensorDataset(x_test, y_test)

#Unpack data
loader = torch.utils.data.DataLoader(xy_train, batch_size=200, shuffle=True)

In [ ]:
losses_train, losses_test = [],[]
for epoch in np.arange(200):
    for x_batch,y_batch in loader:
        net.train() 
        #y_batch = y_batch.type(torch.LongTensor)
        #x_batch = x_batch.type(torch.LongTensor)
        y_pred = net(x_batch)
        loss = loss_func(y_pred,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    losses_train.append(loss.data)
        
    net.eval()
    y_test = y_test.type(torch.LongTensor)
    y_pred = net(x_test)
    loss = loss_func(y_pred,y_test)
    losses_test.append(loss.data)

In [ ]:
plt.plot(losses_train, '.', label="TRAIN (B=200,L.R.=0.1)")
plt.plot(losses_test, '.',label="TEST")
plt.legend()
plt.xlabel('Training Epoch')
plt.ylabel('CrossEntropyLoss')
plt.title("Loss vs Epoch")
plt.yscale('log')

In [ ]:
y_pred = net(x_test)

In [ ]:
y_pred

In [ ]:
y_test

### Possible X-Train, X-Test, Y-Test? 

I'm still unsure about what I can use as the x-train, x-test, and y-test. I'm just trying out different possibilities here. 

One possible x-train I'm thinking about is to select two Dec values and classified each datapoint based on those Dec values then shuffle it. Then train it using y-train.

In [ ]:
import random
lim_0 = 30.5 #1st limit
lim_1 = 30.7 #2nd limit
x_train = []
for Dec in df["DEC"]:
    if Dec > lim_1:
        x_train.append(0)
    elif Dec < lim_0:
        x_train.append(2)
    else:
        x_train.append(1)
random.shuffle(x_train)

In [ ]:
import math
from math import pi, cos, sin

#Display Data
fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)")
ax.set_ylabel("Declination (deg)")
ax.set_title("Randomized")
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,31)
#ax.set_xlim(23,24)
#ax.set_ylim(30.2,31.2)
ax.invert_xaxis()
#ax.set_aspect(1/0.861)
ax.set_aspect("equal")
plt.grid(True,linestyle="--")

# Color code each regions
ax.scatter(df["RA"][np.where(np.array(x_train)==0)[0]],df["DEC"][np.where(np.array(x_train)==0)[0]],s=7,c="red",label="North")
ax.scatter(df["RA"][np.where(np.array(x_train)==1)[0]],df["DEC"][np.where(np.array(x_train)==1)[0]],s=7,c="blue",label="South")
ax.scatter(df["RA"][np.where(np.array(x_train)==2)[0]],df["DEC"][np.where(np.array(x_train)==2)[0]],s=7,c="green",label="Center")
plt.legend()

#-------------------------------------

#Fit Ellipse
u= center_RA       #x-position of the center
v= center_Dec    #y-position of the center
a= 0.52     #radius on the x-axis
b= 0.32      #radius on the y-axis
t_rot=math.radians(58) #rotation angle

t = np.linspace(0, 2*pi, 100)
Ell = np.array([a*np.cos(t) , b*np.sin(t)])  
     #u,v removed to keep the same center location
R_rot = np.array([[cos(t_rot) , -sin(t_rot)],[sin(t_rot) , cos(t_rot)]])  
     #2-D rotation matrix

Ell_rot = np.zeros((2,Ell.shape[1]))
for i in range(Ell.shape[1]):
    Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

plt.plot(u+Ell_rot[0,:] , v+Ell_rot[1,:],c='black',linewidth=0.8)    #rotated ellipse

#Semi-minor Axis
x_SMi_0 = center_RA + (b*cos(np.radians(32))) #x-coor of left Semi-minor axis point 
y_SMi_0 = center_Dec - (b*sin(np.radians(32))) #y-coor of left Semi-minor axis point
x_SMi_1 = center_RA - (b*cos(np.radians(32))) #x-coor of right Semi-minor axis point
y_SMi_1 = center_Dec + (b*sin(np.radians(32))) #y-coor of right Semi-minor axis point
plt.scatter(x_SMi_0,y_SMi_0,s=15,c="black")
plt.scatter(x_SMi_1,y_SMi_1,s=15,c="black")

m_SMi = (y_SMi_1-y_SMi_0)/(x_SMi_1-x_SMi_0) #slope of line through Semi-minor axis 
b_SMi = y_SMi_0 - (m_SMi*x_SMi_0) #y-inter of line through Semi-minor axis
x_SMi = np.linspace(x_SMi_1,x_SMi_0,1000) #x values of line through Semi-minor axis
y_SMi = linear_func(x_SMi,m_SMi,b_SMi) #y values of line through Semi-minor axis
plt.plot(x_SMi,y_SMi,c='black',linewidth=0.8,linestyle="--")

#Semi-major Axis
x_SouthSMa = center_RA - (a*np.sin(np.radians(32))) #x-coor of North Semi-major axis point
y_SouthSMa = center_Dec - (a*np.cos(np.radians(32))) #y-coor of Nouth Semi-major axis point
x_NorthSMa = center_RA + (a*np.sin(np.radians(32))) #x-coor of South Semi-major axis point
y_NorthSMa = center_Dec + (a*np.cos(np.radians(32))) #y-coor of South Semi-major axis point
plt.scatter(x_NorthSMa,y_NorthSMa,c="black",s=15)
plt.scatter(x_SouthSMa,y_SouthSMa,c="black",s=15)

m_SMa = (y_NorthSMa-y_SouthSMa)/(x_NorthSMa-x_SouthSMa) #slope of line through Semi-major axis
b_SMa = y_SouthSMa - (m_SMa*x_SouthSMa) #y-inter of line through Semi-major axis 
x_SMa = np.linspace(x_NorthSMa,x_SouthSMa,1000) #x values of line through Semi-major axis
y_SMa = linear_func(x_SMa,m_SMa,b_SMa) #y values of line through Semi-major axis
plt.plot(x_SMa,y_SMa,c='black',linewidth=0.8,linestyle="--")

NorthPara_XCoor = x_NorthSMa - ((19*a/20)*np.sin(np.radians(32))) #coordinates to move North parabola to
NorthPara_YCoor = linear_func(NorthPara_XCoor,m_SMa,b_SMa)

SouthPara_XCoor = x_SouthSMa + ((19*a/20)*np.sin(np.radians(32))) #coordinate to move South parabola to
SouthPara_YCoor = linear_func(SouthPara_XCoor,m_SMa,b_SMa)

#Define two limits
plt.hlines(30.5,23.2,23.8,linestyle="--",linewidth=1)
plt.hlines(30.7,23.2,23.8,linestyle="--",linewidth=1)

fig.set_figwidth(5)
fig.set_figheight(5)

Would I just take a subset of x_train and y_train and use them as x_test and y_test? 

In [ ]:
x_test = x_train[800:1000] #Possible x_test?
y_test = y_train[800:1000] #Possible y_test?